In [3]:
# Project 3
# Joed Quaye
# Ronald Chomnou
# Griffin Araujo

import pandas as pd
import matplotlib.pyplot as plt
import numpy
from plotnine import ggplot, aes, geom_violin, labs
import plotly.express as px


data = pd.read_csv("gap.tsv", sep='\t')

data.head()

# **Exercise 1**: *Make a scatter plot of life expectancy across time.*

%matplotlib inline
plt.plot(data['year'], data['lifeExp'], "^")

# ggplot is depracated so using plotly
data = pd.read_csv("gap.tsv", sep='\t')

# creating the violin plot using Plotly Express
fig = px.violin(data, x='year', y='lifeExp', title="Life expectancy over time",
                labels={"year": "Year", "lifeExp": "Life Expectancy"})
fig.show()


ModuleNotFoundError: No module named 'plotnine'

In [ ]:
# **Question 2**: How would you describe the distribution of life expectancy across countries 
# for individual years? Is it skewed, or not? Unimodal or not? Symmetric around it’s center?

# During the the earlier years such as years 1952-1982, it is skewed towards the bottom. This means the life expectancy is 
# lower during these years in most countries.

# Question 3


# Question 4


# Question 5



In [ ]:
# Exercise 2

import numpy as np
from sklearn import linear_model
import statsmodels.api as sm


reg = linear_model.LinearRegression()
e2_X = [[x] for x in data['year'].values]
e2_Y = [[y] for y in data['lifeExp'].values]
regfit = reg.fit(e2_X, e2_Y)

X = data['year']  # Independent variable (year)
y = data['lifeExp']  # Dependent variable (life expectancy)

X = sm.add_constant(X)

# fitting the linear regression model
model = sm.OLS(y, X).fit()

# Getting the summary statistics
print(model.summary())

In [ ]:
# Question 6

# On average, life expectancy increases by about .3259 years every year

# Question 7 (MIGHT NEED TO CHECK THIS)

# The null hypothesis of no relationship between year and life expectancy is indeed rejected because of the p value. 
# With each passing year, life expectancy changes. 

In [ ]:
# Exercise 3

pref_life = []
for x in data['year'].values:
    # print(regfit.predict(x)[0][0])
    # reshaping the input to a 2D array
    x_reshaped = np.array(x).reshape(-1, 1)
    
    # Predict life expectancy and append to the list
    pref_life.append(regfit.predict(x_reshaped)[0][0])

data['pred_lifeExp'] = pd.Series(pref_life, index=data.index)
data['residual'] = data['lifeExp'] - data['pred_lifeExp']


plt.figure(figsize=(10, 6))
plt.violinplot(data['residual'], showmeans=True)
plt.title('Violin Plot of Residuals')
plt.xlabel('Year')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# Exercise 4
plt.figure(figsize=(10, 6))
plt.xticks(rotation=45)
plt.title('Boxplot of Residuals by Continent')
plt.xlabel('Continent')
plt.ylabel('Residuals')
plt.boxplot([data[data['continent'] == c]['residual'] for c in data['continent'].unique()], labels=data['continent'].unique())
plt.show()

In [ ]:
#Question 9 Yes there's a dependence between model residual and continent because the distribution of residuals varies  across different continents
#It suggests that there may be other effect that impact the of time (or other independent variables) on life expectancy differs across continents. 
#It Suggest that there may be additional factors specific to each continent that are influencing life expectancy beyond what is captured by 
#the current regression model. So we have to employ more sophisticated modeling techniques to better capture the nuances of the relationship between 
#residual and continent.

In [ ]:
from plotnine import ggplot, aes, geom_point, geom_line, theme_bw, stat_smooth, scale_color_manual

color_palette = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]

(ggplot(data=data) +
 aes(x='year', y='lifeExp', color='continent') +
 stat_smooth(method="lm", se=True, span=0.2) +
 geom_point(size=1) + 
 scale_color_manual(values=color_palette) + 
 theme_bw())

In [ ]:
# Question 10 Given the uniform increase in life expectancy observed across all continents over time 
# and the absence of notable disparities in trends between them, there's not much of a reason  for including 
# an interaction term for continent and year in the regression model. Opting for a simpler model without this 
# interaction term offers easier interpretation especially if the dataset is not notably large.

In [ ]:
import statsmodels.api as sm

# Fit a linear regression model with an interaction term for continent and year
model = sm.OLS.from_formula('lifeExp ~ year + continent + year:continent', data=data).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
pip install scikit-learn

In [ ]:
# Exercise 6
import statsmodels.api as sm

# Fit a linear regression model with an interaction term for continent and year
model = sm.OLS.from_formula('lifeExp ~ year + continent + year:continent', data=data).fit()

# Print the summary of the regression model
print(model.summary())

#Question 11
# Not all of the parameters in the model are significantly differently from zero.
# Asia, the Americas, and Europe are significantly different from zero because their
# p-values(P>|t|) are less than zero
# Oceania is not significantly different from zero because the p-values(P>|t|) is bigger than zero

# Question 12
# Extract coefficient estimates for 'year' and its interactions with each continent
year_coef = model.params['year']
interactions_coef = {name: model.params[name] for name in model.params.index if 'year:C' in name}

# Calculate the average increase in life expectancy each year for each continent
average_increase = {}
for continent, interaction_coef in interactions_coef.items():
    average_increase[continent.split("[T.")[-1][:-1]] = year_coef + interaction_coef

# Print the results
for continent, increase in average_increase.items():
    print(f"On average, life expectancy increases by {increase:.2f} years each year in {continent}.")



In [ ]:
# Exercise 7
## Ftest for Exercise 2
model2 = sm.OLS.from_formula(formula='lifeExp ~ year', data=data).fit()
table2 = sm.stats.anova_lm(model2, typ=2)
print("Exercise 2")
print(table2)
## Ftest for Exercise 6
table = sm.stats.anova_lm(model, typ=2)
print("Exercise 6")
print(table)

# Question 13  Is the interaction model significantly better than the year-only model? Why?
# Yes, the interaction model is significantly better than the year-only model.
# It is significantly better because of the associated p-values. The p-value is 6.463379e-14 as seen
# in the table which is much less than 0.05.

In [ ]:
# Exercise 8
import matplotlib.pyplot as plt

residuals = model.resid

# Create a violin plot of residuals vs. year
fig = px.violin(x=data['year'], y=residuals, box=True, points="all")
fig.update_traces(meanline_visible=True)
fig.update_layout(
    title='Residuals vs. Year (Interaction Model)',
    xaxis_title='Year',
    yaxis_title='Residuals'
)
fig.show()
#Spread of residuals appears mostly consistent meaning constant variance except for the few outliers
# on each end

# Create a residuals vs. fitted values plot
plt.figure(figsize=(10, 6))
plt.scatter(model.fittedvalues, residuals, alpha=0.5)
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs. Fitted Values (Interaction Model)')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()
#There doesn't seem to be obvious patterns visible
# Even spread of points at y = 0 though meaning constant variance

